## Test for sliding windows on an image

In [ ]:
output_img = np.dstack((binary_warped * 255, binary_warped * 255, binary_warped * 255))

window_width = 50
margin = 100
window_height = 80
window = np.ones(window_width)

l_center, r_center = lane_algo.find_bottom_left_right()
l_center = r_center
start_x = 640

boxes_y = np.arange(0, binary_warped.shape[0], lane_algo.window_height)[::-1]
centers = np.zeros_like(boxes_y)
conv_max = np.zeros_like(boxes_y)

# set first center to the last one as the loop uses last center
centers[-1] = l_center

for i in range(0, len(boxes_y)):
    y = boxes_y[i]

    margin_lx = max(centers[i - 1] - window_width // 2 - margin // 2, 0)
    margin_rx = min(centers[i - 1] + window_width // 2 + margin // 2, binary_warped.shape[1])
    
    lt = margin_lx, y
    rb = margin_rx, y + window_height
    
    cv2.rectangle(output_img, lt, rb, (0, 255, 0), 10)

    image_area = binary_warped[y: y + window_height, margin_lx : margin_rx]
    
    sum_cols = np.sum(image_area, axis=0)
    conv = np.convolve(window, sum_cols, 'same')
    
    f, (ax1, ax2) = plt.subplots(1,2, figsize=(20,5))
    ax1.plot(conv)
    ax1.set_title('Conv Sum')
    ax2.imshow(output_img)
    plt.show()
    
    conv_max[i] = np.argmax(conv)

    if conv_max[i] == 0:
        centers[i] = centers[i - 1]
    else:
        centers[i] = conv_max[i] + margin_lx
        #print('Conv Max', conv_max[i], 'center', centers[i])

        # how many pixels were there in this box compared to the previous one
        # if not many then don't believe so much on this box

        if i > 0:
            div = (conv_max[i - 1]) + 1e-4
            filled_area_ratio = abs(conv_max[i] / div)

            if filled_area_ratio < 0.3:
                center_diff = (centers[i] - centers[i-1]) / window_width
                #print('This', centers[i], 'Prev', centers[i-1], 'Diff:', center_diff, 'Filled:', filled_area_ratio)

                if center_diff > 0.2:
                    centers[i] = np.int(centers[i-1] * -0.8)
                elif center_diff < -0.2:
                    centers[i] = np.int(centers[i-1] * 0.8)

                #print('This', centers[i], 'Prev', centers[i-1], 'Diff:', center_diff, 'Filled:', filled_area_ratio)

                # carry forward 80% weight of the previous conv_max as we go up we don't want
                # one or two pixels in this window to allow another small ratio of pixels to be impacted
                conv_max[i] = conv_max[i-1] * 0.8
        
    lt = centers[i] - window_width // 2, y
    rb = centers[i] + window_width // 2, y + window_height

    cv2.rectangle(output_img, lt, rb, (255, 0, 0), 10)
    
plt.imshow(output_img)
plt.show()